# Assignment 2 - Feature extraction and classification

Note: This notebook file for the assignment has deviations from the course guide with respect to the structure, sentence framing, question framing and numbering. Please consider this notebook file structure as the final structure and follow this.

In this assignment, you are expected to

(1) extract global features from CIFAR10 dataset with one of the pre-trained neural networks available in pytorch,

(2) classify the dataset using the traditional k-Nearest Neighbours classifier,

and

(3) implement k-fold cross-validation to evaluate your model.

------------------------

In [1]:
# Load all the needed packages for this assignment here
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

/Users/hungnguyen/miniforge3/envs/DS-CVassignment/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/hungnguyen/miniforge3/envs/DS-CVassignment/lib/python3.10/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE'
  warn(f"Failed to load image Python extension: {e}")


When working with Pytorch, dataloader() is a must to know function. Read more about this function and the parameters it accepts in https://blog.paperspace.com/dataloaders-abstractions-pytorch/

In [2]:
from torch.utils.data import DataLoader

The variable 'transform' encapsulates the needed transformations of our data. Read more about transforms in https://blog.paperspace.com/dataloaders-abstractions-pytorch/

In [3]:
transform = transforms.Compose([
    # resize
    transforms.Resize(32),
    # center-crop
    transforms.CenterCrop(32),
    # to-tensor
    transforms.ToTensor(),
    # normalize
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

### INPUT DATASET

Load the CIFAR10 dataset from Pytorch

In [4]:
dataset = torchvision.datasets.CIFAR10(root='cifar10', train=True, download=False, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)
dataset_test = torchvision.datasets.CIFAR10(root='cifar10', train=False, download=False, transform=transform)

#### Exercise 2.1 - Dataset preparation

**a)** Write a function **'train_test_split(dataset, ratio)'** which takes a dataset array as an input and returns two dataset arrays- one for training and another for testing.


In [12]:
from sklearn.model_selection import train_test_split
import pandas as pd


### FEATURE EXTRACTION

Extract descriptors from the images in your train and test dataset. The dataset split should remain the same for all the experiments if you want to be fair when comparing performance.

#### Exercise 2.2 - Feature 1 - RGB descriptor

Implement the same code you wrote for extracting the overall RGB descriptors(of size n x 24) as in assignment 1 here.

In [14]:
# Ex.2.2 your code here

def extract_rgb_descriptors(datasets):
    dataset_list = []
    for image, label in datasets:
        image_feature = []
        # image_feature_with_label = []
        colors = ("red", "green", "blue")
        image = np.array(image)
        # print(image.shape)
        # plt.imshow(image)
        # break
        # image_feature_with_label.append(label)
        for channel_id, color in enumerate(colors):
            histogram, bin_edges = np.histogram(
                image[:, :, channel_id], bins=8, range=(0, 256)
            )
            image_feature = np.concatenate((image_feature, histogram))
        # print(image_feature)
        image_feature = image_feature / 784
        image_feature = np.append(image_feature,label)
        dataset_list.append(image_feature)
    # training_data, val_data, train_label, val_label = train_test_split(train_df, label_list, test_size=0.2, random_state=42)
    return pd.DataFrame(dataset_list)

rgb_descriptors_df = extract_rgb_descriptors(dataset)

#### Exercise 2.3 - Feature 2 - Extract CNN descriptors using pre-traind networks

Load one of the pretrained network (resnet, alexnet, vgg, squeezenet, densenet, inception) from pytorch to extract global features from the images present in the dataset. 
We will use the output values from the layer present just before the fully connected layer of the deep network as a descriptor, i.e. we will remove the last fully-connected layer. Therefore, after feed-forwarding the input image through the network, we save the output as the descriptor of the image. We do this for all the images present in the dataset to get the overall CNN descriptors.

You may refer to this link for debugging purposes - https://stackoverflow.com/questions/52548174/how-to-remove-the-last-fc-layer-from-a-resnet-model-in-pytorch

In [19]:
# Ex.2.3 your code here
# name of the model you wish to use - it should be selected from this list
# [resnet, alexnet, vgg, squeezenet, densenet, inception]

import torch.nn as nn
from torchvision.models import resnet18
model = resnet18(weights=True)
model_without_fc = torch.nn.Sequential(*(list(model.children())[:-1]))
# out = model_without_fc(features_df)


/Users/hungnguyen/miniforge3/envs/DS-CVassignment/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[0.0727, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0204, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1110, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.1224, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1071, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0038, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

### PERFORMANCE EVALUATION

#### Exercise 2.4 - Error function

Implement a function to evaluate the accuracy of your prediction. We will rely on the evaluation metric 'accuracy'.

You are suggested to also use f-score, recall and precision. Have a look at https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html 

In [ ]:
def accuracy_metric(actual, predicted):
    
    # Ex.2.4 your code here
    
    return accuracy_value

### TRAIN AND TEST YOUR MODEL

#### Exercise 2.5 - k Nearest Neighbour model

For this exercise, first split the extracted overall RGB and CNN descriptor to train and test sets with the help of the 'train_test_split()' function that you implemented before.

**a)** Apply the classifier with different values of k (number of nearest neighbours) to the train **RGB descriptor** set and evaluate the performance of your models using the accuracy_metric() function that you implemented before.

You can have a look at the documentation to understand the parameters that define the learning of the model,
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Use your k-NN - play with the value of the parameters to see how the model performs
kvalue_list = [2,4,6,10,15] 

# Ex.2.5a your code here

**b)** Apply the classifier with different values of k (number of nearest neighbours) to the train **CNN descriptor** and evaluate the performance of your models using the accuracy_metric() function that you implemented before.

In [ ]:
# Ex.2.5b your code here

#### Exercise 2.6 - Visualize results 

**a)** Since you already applied PCA to the extracted overall RGB descriptor in assignment 1, now apply PCA to the extracted overall **CNN descriptor**.

Steps to follow:

1) Choose the kNN classifier with k value that gave you the best results in the previous exercise and use it to make predictions on your train CNN descriptor set.

2) Apply PCA on the train set and select the first 2 principal components to represent each sample.

2) Plot the principal components representing the samples with empty circles. Use one color per ground truth class lables. On top of this, plot the samples again but now with filled circles. For these filled circles, use the color of the class predicted per sample in step 1. You can note that misclassifications will make the colours not coincide.

In [ ]:
# Ex.2.6a your code here

**b)** Repeat the steps mentioned before but now on the test CNN descriptor set. 

In [ ]:
# Ex.2.6b your code here

#### Exercise 2.7 - kNN with k-Fold cross-validation

Assess the performance of your implemented kNN using k-Fold cross-validation. 

Run your implemented function evaluating for k (fold) = 2, 5 and 10. You can rely on the kNN that performed best in the previous exercises.
Report the average accuracy and the standard deviation.

In [ ]:
# Load packages
from sklearn.model_selection import KFold
import numpy as np
from sklearn.utils import shuffle

# Ex.2.7 your code here

In [ ]:
## SUGGESTION ON HOW TO PRESENT PERFORMANCE OF YOUR KFOLD CROSS VALIDATION ANALYSIS

print('Summary results:')
print(' ')
print(' ')
for i,k in enumerate(k_list):
    print(k,'-fold cross validation:')  
    print('Accuracies per fold: ', avg_acc_list[i]) 
    
    avg_acc = round(sum(avg_acc_list[i])/k,2)
    std_list= round(np.std(avg_acc_list[i]),2)
    print('Average accuracy: ', avg_acc,'+-', std_list) 
    print(' ')

### [Optional] Exercise: further explore by: 
- implement other classifiers such as SVM or Random Forest, 
- extract other descriptors from the images such as objects or other local features,
- implement the evaluation metrics: recall, precision and f-score.